In [1]:
import requests
import pandas as pd
import numpy as np

In [39]:
def divide_chunks(n, d=100):
    return int((n+ (d-1))/d) + 1

In [42]:
base_url = "https://api.fabdb.net/cards"

payload = {
    "page": 1,
    "per_page": 100,
}

pages = []
for i in range(1, divide_chunks(656, 100)):
    r = requests.get(base_url, params=payload)
    pages.append(r.json())
    print(r.json()['meta'])
    payload['page'] = i+1

{'current_page': 1, 'from': 1, 'last_page': 7, 'path': '/cards', 'per_page': '100', 'to': 100, 'total': 656}
{'current_page': 2, 'from': 101, 'last_page': 7, 'path': '/cards', 'per_page': '100', 'to': 200, 'total': 656}
{'current_page': 3, 'from': 201, 'last_page': 7, 'path': '/cards', 'per_page': '100', 'to': 300, 'total': 656}
{'current_page': 4, 'from': 301, 'last_page': 7, 'path': '/cards', 'per_page': '100', 'to': 400, 'total': 656}
{'current_page': 5, 'from': 401, 'last_page': 7, 'path': '/cards', 'per_page': '100', 'to': 500, 'total': 656}
{'current_page': 6, 'from': 501, 'last_page': 7, 'path': '/cards', 'per_page': '100', 'to': 600, 'total': 656}
{'current_page': 7, 'from': 601, 'last_page': 7, 'path': '/cards', 'per_page': '100', 'to': 656, 'total': 656}


In [43]:
dfs = []
for i in pages:
    dfs.append(pd.DataFrame.from_dict(i['data']))
df = pd.concat(dfs).reset_index()

In [44]:
stat_list = df.stats.values.tolist()
for n, i in enumerate(stat_list):
    if i == []:
        stat_list[n] = {}
        
df = df.drop(['stats', 'index'], axis=1).join(pd.DataFrame(stat_list))
# numeric_cols = ['resource', 'attack', 'cost', 'defense', 'intellect', 'life']
# df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [45]:
def transform_name(row):

    resource_map = {
        '3': '(Blue)',
        '2': '(Yellow)',
        '1': '(Red)',
        np.nan: '',
    }
    
    return f"{row['name']} {resource_map[row['resource']]}"

In [46]:
df['name_resource'] = df.apply(transform_name, axis=1)

In [47]:
df = df[~df['identifier'].str.contains("IRA")]
df = df.drop_duplicates('name_resource')


In [48]:
df.to_csv("cards.csv", index=False)

In [49]:
df[df['name'] == "Sink Below"]

,identifier,image,keywords,name,rarity,text,variants,attack,cost,defense,intellect,life,resource,name_resource
645,WTR215,https://fabdb2.imgix.net/cards/wtr/215.png?w=3...,"[generic, defense, reaction]",Sink Below,C,You may put a card from your hand on the botto...,[],NaN,0,4,NaN,NaN,1,Sink Below (Red)
646,WTR216,https://fabdb2.imgix.net/cards/wtr/216.png?w=3...,"[generic, defense, reaction]",Sink Below,C,You may put a card from your hand on the botto...,[],NaN,0,3,NaN,NaN,2,Sink Below (Yellow)
647,WTR217,https://fabdb2.imgix.net/cards/wtr/217.png?w=3...,"[generic, defense, reaction]",Sink Below,C,You may put a card from your hand on the botto...,[],NaN,0,2,NaN,NaN,3,Sink Below (Blue)


In [50]:
df.shape

(619, 14)